In [1]:
from joblib import load
from os import path
import random
import pandas as pd

In [2]:
dummy = True

In [3]:
task_name = "crabnet_hyperparameter"
model_dir = path.join("..", "..", "models", task_name)

if dummy:
    model_dir = path.join(model_dir, "dummy")
   

In [4]:
class CrabNetSurrogateModel(object):
    def __init__(self):
        self.models = load(path.join(model_dir, "surrogate_models.pkl"))

    def prepare_params_for_eval(self, raw_params):
        raw_params["bias"] = int(raw_params["bias"])
        raw_params["use_RobustL1"] = raw_params["criterion"] == "RobustL1"
        raw_params.pop("criterion")

        raw_params.pop("losscurve")
        raw_params.pop("learningcurve")

        # raw_params["train_frac"] = random.uniform(0.01, 1)

        elem_prop = raw_params["elem_prop"]
        raw_params["elem_prop_magpie"] = 0
        raw_params["elem_prop_mat2vec"] = 0
        raw_params["elem_prop_onehot"] = 0
        raw_params[f"elem_prop_{elem_prop}"] = 1
        raw_params.pop("elem_prop")

        return raw_params

    def surrogate_evaluate(self, params):

        parameters = self.prepare_params_for_eval(params)
        parameters = pd.DataFrame([parameters])

        percentile = random.uniform(0, 1)  # generate random percentile

        mae = self.models["mae"].predict(parameters.assign(mae_rank=[percentile]))
        rmse = self.models["rmse"].predict(parameters.assign(rmse_rank=[percentile]))
        runtime = self.models["runtime"].predict(
            parameters.assign(runtime_rank=[percentile])
        )
        model_size = self.models["model_size"].predict(parameters)

        return mae, rmse, runtime, model_size

In [5]:
parameterization = {
    "N": 3,
    "alpha": 0.5,
    "d_model": 512,
    "dim_feedforward": 2048,
    "dropout": 0.1,
    "emb_scaler": 1.0,
    "epochs_step": 10,
    "eps": 0.000001,
    "fudge": 0.02,
    "heads": 4,
    "k": 6,
    "lr": 0.001,
    "pe_resolution": 5000,
    "ple_resolution": 5000,
    "pos_scaler": 1.0,
    "weight_decay": 0,
    "batch_size": 32,
    "out_hidden4": 128,
    "betas1": 0.9,
    "betas2": 0.999,
    "losscurve": False,
    "learningcurve": False,
    "bias": False,
    "criterion": "RobustL1",
    "elem_prop": "mat2vec",
    "train_frac": 0.5,
}

In [6]:
c = CrabNetSurrogateModel()

In [7]:
mae, rmse, runtime, model_size = c.surrogate_evaluate(parameterization)
print(f"MAE: {mae}, RMSE: {rmse}, runtime: {runtime}, model_size: {model_size}")

MAE: [1.01922182], RMSE: [1.47120058], runtime: [265.26229555], model_size: [14660828.53]


c:\Users\sterg\Miniconda3\envs\matsci-opt-benchmarks\lib\site-packages\sklearn\base.py:488: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names must be in the same order as they were in fit.

  warnings.warn(message, FutureWarning)
c:\Users\sterg\Miniconda3\envs\matsci-opt-benchmarks\lib\site-packages\sklearn\base.py:488: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names must be in the same order as they were in fit.

  warnings.warn(message, FutureWarning)
c:\Users\sterg\Miniconda3\envs\matsci-opt-benchmarks\lib\site-packages\sklearn\base.py:488: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names must be in the same order as they were in fit.

  warnings.warn(message, FutureWarning)
c:\Users\sterg\Miniconda3\envs\mat

In [147]:


# common_features = [
#     "N",
#     "alpha",
#     "d_model",
#     "dim_feedforward",
#     "dropout",
#     "emb_scaler",
#     "eps",
#     "epochs_step",
#     "fudge",
#     "heads",
#     "k",
#     "lr",
#     "pe_resolution",
#     "ple_resolution",
#     "pos_scaler",
#     "weight_decay",
#     "batch_size",
#     "out_hidden4",
#     "betas1",
#     "betas2",
#     "train_frac",
#     "bias", # - 0,1, use int(bool)
#     "use_RobustL1", #
#     # "criterion_RobustL2",
#     "elem_prop_magpie", #
#     "elem_prop_mat2vec", #
#     "elem_prop_onehot", #
#     # "hardware_2080ti"
# ]